# spectre Benchmarks

In [1]:
import pandas as pd
print(pd.__version__)
start, end = pd.Timestamp('2013-01-02', tz='UTC'), pd.Timestamp('2018-01-02', tz='UTC')

import sys
sys.path.append('..\\..\\spectre')

from spectre import factors, parallel
import pandas as pd
loader = factors.ArrowLoader('../../historical_data/us/prices/quandl/wiki_prices.feather')

0.25.2


In [2]:
engine = factors.FactorEngine(loader)
engine.to_cuda()

In [3]:
engine.remove_all_factors()
engine.add(factors.MA(100), 'ma')
%time dfsp=engine.run(start, end)
dfsp.head(5).append(dfsp.tail(5))

Wall time: 1.5 s


ma
date                      asset           
2013-01-03 00:00:00+00:00 A      38.110535
                          AAL    11.776899
                          AAMC         NaN
                          AAN    28.779516
                          AAON   19.606133
2018-01-02 00:00:00+00:00 ZIXI    4.834800
                          ZNGA    3.828900
                          ZOES   13.509800
                          ZTS    66.429054
                          ZUMZ   17.311750

In [4]:
%timeit engine.run(start, end)
%timeit -n 3 -r 10 engine.run(start, end)

96.4 ms ± 14.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
87.4 ms ± 609 µs per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [5]:
engine.remove_all_factors()
engine.add(factors.EMA(50, adjust=True), 'ema50')
%time dfsp=engine.run(start, end)
dfsp.head(5).append(dfsp.tail(5))

Wall time: 499 ms


ema50
date                      asset           
2013-01-03 00:00:00+00:00 A      38.895058
                          AAL    12.636659
                          AAMC         NaN
                          AAN    28.475159
                          AAON   20.192312
2018-01-02 00:00:00+00:00 ZIXI    4.589237
                          ZNGA    3.923355
                          ZOES   15.105145
                          ZTS    69.987251
                          ZUMZ   19.599936

In [6]:
%timeit engine.run(start, end)
%timeit -n 3 -r 10 engine.run(start, end)

154 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
144 ms ± 1.14 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [7]:
engine.remove_all_factors()
engine.add(factors.MACD(), 'MACD')
%time dfsp=engine.run(start, end)
dfsp.head(5).append(dfsp.tail(5))

Wall time: 153 ms


MACD
date                      asset          
2013-01-03 00:00:00+00:00 A      0.871926
                          AAL    0.231277
                          AAMC        NaN
                          AAN   -0.245126
                          AAON   0.061136
2018-01-02 00:00:00+00:00 ZIXI  -0.052928
                          ZNGA   0.019368
                          ZOES   0.898339
                          ZTS    0.850278
                          ZUMZ   0.593955

In [8]:
%timeit engine.run(start, end)
%timeit -n 3 -r 10 engine.run(start, end)

145 ms ± 731 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
145 ms ± 1.94 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [9]:
engine.remove_all_factors()
f = factors.MACD()+factors.RSI()+factors.STOCHF()
engine.add(f.rank().zscore(), 'f')
%time dfsp=engine.run(start, end)
dfsp.head(5).append(dfsp.tail(5))

Wall time: 234 ms


f
date                      asset          
2013-01-03 00:00:00+00:00 A      0.179031
                          AAL    0.999540
                          AAMC        NaN
                          AAN    0.138977
                          AAON  -0.495825
2018-01-02 00:00:00+00:00 ZIXI   0.068631
                          ZNGA   0.941732
                          ZOES  -0.354476
                          ZTS   -0.449286
                          ZUMZ  -0.036084

In [10]:
%timeit engine.run(start, end)
%timeit -n 3 -r 10 engine.run(start, end)

294 ms ± 62.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
375 ms ± 65.4 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [11]:
import torch
torch.cuda.max_memory_allocated()/1024**3, 'GB'

(0.9656634330749512, 'GB')